Morgan Turville-Heitz
CS 760
10/18/2023

In [16]:

import pandas as pd
import numpy as np
import os
import ipykernel

Defining the Doc class, where counts/vectors are stored for each document.

In [17]:
class docRegistry(type):
    def __iter__(cls):
        return iter(cls._registry)
    

In [18]:
class Doc(metaclass=docRegistry):
    _registry = []

    def __init__(self, raw, fn):
        self._registry.append(self)
        self.fn = fn
        self.vector = self.vectorize(raw)
        self.char_counts = self.count(self.vector)
        self.label = fn[0]
        self.pe = None
        self.pj = None
        self.ps = None

    def vectorize(self, data):
        ### Stripping newlines/special characters
        filtered = ''.join([char for char in data if char.isalpha() or char == ' '])
        return list(filtered)

    def count(self, vector):
        count = {char: 0 for char in 'abcdefghijklmnopqrstuvwxyz '}
        for char in vector:
            count[char]+=1
        return count


Loading files, creating the Doc class for each file.

In [19]:
direc = r"C:\Users\Meau\Documents\GRAD SCHOOL\CS 760\Hw4\languageID"
files = os.listdir(direc)
for txtfile in files:
    path = os.path.join(direc, txtfile)
    with open(path, 'r') as file:
        raw = file.read()
    Doc(raw, str(txtfile))

Calculating the priors, with additive smoothing $$ P(x) = \frac{n(x) + \alpha}{N + k \alpha }, \alpha = \frac{1}{2} $$ 


In [25]:
counts = {"e":0, "s":0, "j":0}
logpriors = {"e":0, "s":0, "j":0}
alpha = 0.5
Nd = 0
k = 3
for doc in Doc:
    Nd += 1
    lang = doc.label
    counts[lang] += 1

for la in counts.keys():
    logpriors[la] = np.log((counts[la] + alpha ) / (Nd + k * alpha))


priors = {la : np.exp(logpriors[la]) for la in logpriors.keys()}

priors

{'e': 0.3333333333333333, 's': 0.3333333333333333, 'j': 0.3333333333333333}